In [ ]:
# Team 3 - Ctrl C; Ctrl V
import os
import csv
os.chdir('/kaggle/working')
with open('submissions.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar = "|", quoting = csv.QUOTE_MINIMAL)
    filewriter.writerow(['id', 'xmin', 'xmax', 'ymin', 'ymax'])
    
cIdsA = []
cIdsA.append('a_43')
cIdsA.append('a_32')
cIdsA.append('a_47')
cIdsA.append('a_48')
cIdsA.append('a_40')
cIdsA.append('a_28')
cIdsA.append('a_37')
cIdsA.append('a_27')
cIdsA.append('a_30')
cIdsA.append('a_36')
cIdsA.append('a_45')
cIdsA.append('a_25')
cIdsA.append('a_38')
cIdsA.append('a_35')
cIdsA.append('a_46')

cIdsB = []
cIdsB.append('b_25')
cIdsB.append('b_39')
cIdsB.append('b_42')
cIdsB.append('b_35')
cIdsB.append('b_36')
cIdsB.append('b_43')
cIdsB.append('b_34')
cIdsB.append('b_44')
cIdsB.append('b_26')
cIdsB.append('b_38')
cIdsB.append('b_29')
cIdsB.append('b_33')
cIdsB.append('b_31')
cIdsB.append('b_41')

cIdsC = []
cIdsC.append('c_47')
cIdsC.append('c_12')
cIdsC.append('c_0')
cIdsC.append('c_4')
cIdsC.append('c_9')
cIdsC.append('c_14')
cIdsC.append('c_15')
cIdsC.append('c_22')
cIdsC.append('c_11')
cIdsC.append('c_23')
cIdsC.append('c_2')
cIdsC.append('c_24')
cIdsC.append('c_13')

cIdsD = []
cIdsD.append('d_20')
cIdsD.append('d_19')
cIdsD.append('d_6')
cIdsD.append('d_17')
cIdsD.append('d_10')
cIdsD.append('d_16')
cIdsD.append('d_18')
cIdsD.append('d_21')
cIdsD.append('d_7')
cIdsD.append('d_8')
cIdsD.append('d_3')
cIdsD.append('d_42')
cIdsD.append('d_1')
cIdsD.append('d_5')

In [ ]:
!pip install pycocotools

In [ ]:
#LFW Imports
import os
os.chdir('/kaggle/input/alldata/Project')

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer

from models_lfw.face_recognition.model_small import create_model
from models_lfw.face_recognition.align import AlignDlib
from models_lfw.triplet_loss import TripletLossLayer
from coml_preprocessor import ComlFaceDataGenerator

#DETR Imports
os.chdir('detr')

import argparse
import random
import cv2
from pathlib import Path
import torch
import torchvision.transforms as T
import PIL.Image
from models import build_model
from main import get_args_parser
import sys
import json
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import csv

In [ ]:
mapping = {}
mapping2 = {}
with open('/kaggle/input/mapping/person_name_id_mapping.csv', newline='') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         mapping[row["Name"]] = int(row["ID"])
         mapping2[int(row["ID"])] = row["Name"]  # This stores Id in a key-value pair format similar to a java map

In [ ]:
# Model Configurations
parser = argparse.ArgumentParser(description='DETR args parser', parents=[get_args_parser()])
args = parser.parse_args(args=[])
args.resume = '/kaggle/input/alldata/Project/models_lfw/checkpoint.pth'
args.device = 'cpu'

if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)

args.distributed = False

print(args) 

# Buuld Model
model, criterion, postprocessors = build_model(args)

device = torch.device(args.device)
model.to(device)

# Load Weights
output_dir = Path(args.output_dir)
if args.resume:
    # the model will download the weights and model state from the https link provided
    if args.resume.startswith('https'):
      checkpoint = torch.hub.load_state_dict_from_url(
          args.resume, map_location='cpu', check_hash = True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')

    # this load the weights and model state into the model
    model.load_state_dict(checkpoint['model'], strict=True)

# COCO classes
CLASSES = [
   'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
   'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
   'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
   'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
   'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
   'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
   'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
   'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
   'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
   'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
   'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
   'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
   'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
   'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def detect(im, model, transform):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0)

    assert img.shape[-2] <= 1600 and img.shape[-1] <=1600, 'demo model only supports images up to 1600 pixels on each side'

    outputs = model(img)

    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.7

    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
    return probas[keep], bboxes_scaled

def plot_results(pil_img, prob, boxes, classes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        c1 = p.argmax()
        if CLASSES[c1] not in classes:
            continue
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        
        text= f'Find This Person'
        #f'{CLASSES[c1]}: {p[c1]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=.5))
    plt.axis('off')
    plt.show()

# Stores coordinates of the bounding box
def bbox_dims(pil_img, prob, boxes, classes):
    bbox_list = []
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        c1 = p.argmax()
        if CLASSES[c1] not in classes:
            continue
        bbox_list.append([xmin, ymin, xmax, ymax])
    return bbox_list

In [ ]:
letter = "a"
the_image = PIL.Image.open('/kaggle/input/2021-spring-coml-face-recognition-competition/' + str(letter) + '.jpg')
width, height = the_image.size

if width > 1600 and height > 1600:
  new_size = (1600, 1600)
  the_image = the_image.resize(new_size)

plot_classes = ['person']
scores, boxes = detect(the_image, model, transform)

bbox_dim_list = bbox_dims(the_image, scores, boxes, plot_classes)

# Crop Images, this will be fed into the coml generator
cropped_images = []
for bbox in bbox_dim_list:
    cropped_images.append(the_image.crop((bbox[0], bbox[1], bbox[2], bbox[3]))) # Left, upper, right, bottom

In [ ]:
# Input for anchor, positive, and negative images
in_a = Input(shape=(96, 96, 3), name="img_a")
in_p = Input(shape=(96, 96, 3), name="img_p")
in_n = Input(shape=(96, 96, 3), name="img_n")

# create the base model from model_small
model_sm = create_model()

# Output the embedding vectors from anchor, positive, and negative images
# The model weights are shared (Triplet network)
emb_a = model_sm(in_a)
emb_p = model_sm(in_p)
emb_n = model_sm(in_n)

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Model that can be trained with anchor, positive, and negative images
model = Model([in_a, in_p, in_n], triplet_loss_layer)

# Load weights
model.load_weights("/kaggle/input/alldata/Project/ckpts(lfw)/epoch024_loss1.440.hdf5")

base_model = model.layers[3]

class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path, upper_limit):
    metadata = []
    count = 0
    for i in sorted(os.listdir(path)):
        if count == upper_limit: 
            break
            
        for f in sorted(os.listdir(os.path.join(path, i))):
            if count == upper_limit: 
                break
                
            count += 1
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]
    
def align_image(img):
        alignment = AlignDlib('/kaggle/input/alldata/Project/models_lfw/landmarks.dat')
        bb = alignment.getLargestFaceBoundingBox(img)
        if bb is None:
            return cv2.resize(img, (96,96))
        else:
            return alignment.align(96, 
                                   img, 
                                   bb,
                                   landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [ ]:
metadata = load_metadata('/kaggle/input/alldata/Project/data/fire', 100)
alignment = AlignDlib('/kaggle/input/alldata/Project/models_lfw/landmarks.dat')

embeddings = np.empty((metadata.shape[0], 128))

for i, m in enumerate(metadata):
    img = load_image(m.image_path())
    img = align_image(img)
    img = img.astype('float32')
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    person = m.image_path().split('/')[7].replace('_', ' ')
    embeddings[i] = base_model.predict(img)
    # for displaying the progress of creating embeddings
    if i%1 == 0: print(str(i) + " " + person + "\n", end=" ")
    if i == len(metadata)-1: print('done')

In [ ]:
IdsUsed = []

In [ ]:
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

def getMinimumDistance(myEmbeddings, embeddings, myEmbeddingsIndex):
  name = "IDK"
  dist = .2
  name = [metadata[0].image_path().split('/')[7].replace('_', ' ')] 
  for i in range(len(embeddings)):
    if distance(myEmbeddings[myEmbeddingsIndex], embeddings[i]) < dist:
      name = [metadata[i].image_path().split('/')[7].replace('_', ' ')] 
  return str(name).replace("[", "").replace("]", "").replace("'", "")

def plotWithNameAndId(pil_img, prob, boxes, classes, cIds, theLetter):
    correctCount = 0
    count = 0
    filler = 0
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    i = 0
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        c1 = p.argmax()
        if CLASSES[c1] not in classes:
            continue
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        if (getMinimumDistance(myEmbeddings, embeddings, i) != "IDK"):
            id = mapping[getMinimumDistance(myEmbeddings, embeddings, i)]
        name = str(getMinimumDistance(myEmbeddings, embeddings, i))
        text = str(getMinimumDistance(myEmbeddings, embeddings, i))
        neededID = str(letter) + "_" + str(id)
        theId = id
        i += 1
        coor = (name + ", " + "Id: " + neededID + ", Coordinates: " + str(xmin) + " " + str(xmax) + " " + str(ymin) + " " + str(ymax))
        print(coor)
        
        os.chdir('/kaggle/working')
        with open('submissions.csv', 'a') as csvfile:
            file = open("submissions.csv")
            reader = csv.reader(file)
            filewriter = csv.writer(csvfile, delimiter=',', quotechar = "|", quoting = csv.QUOTE_MINIMAL)
            lines = len(list(reader))
            if neededID in cIds:
                if not neededID in IdsUsed:
                    filewriter.writerow([neededID, int(xmin), int(xmax), int(ymin), int(ymax)])
                    IdsUsed.append(neededID)
                     
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=.5))
    plt.axis('off')
    plt.show()

In [ ]:
myEmbeddings = np.empty((metadata.shape[0], 128))

for i,img in enumerate(cropped_images):
  img = align_image(np.asarray(img))
  img = img.astype('float32')
  img =img / 255.0
  img = np.expand_dims(img, axis=0)
  person = str(i)
  myEmbeddings[i] = base_model.predict(img)

In [ ]:
plotWithNameAndId(the_image, scores, boxes, plot_classes, cIdsA, 'a')

In [ ]:
# Model Configurations
parser = argparse.ArgumentParser(description='DETR args parser', parents=[get_args_parser()])
args = parser.parse_args(args=[])
args.resume = '/kaggle/input/alldata/Project/models_lfw/checkpoint.pth'
args.device = 'cpu'

if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)

args.distributed = False

print(args) 

# Buuld Model
model, criterion, postprocessors = build_model(args)

device = torch.device(args.device)
model.to(device)

# Load Weights
output_dir = Path(args.output_dir)
if args.resume:
    # the model will download the weights and model state from the https link provided
    if args.resume.startswith('https'):
      checkpoint = torch.hub.load_state_dict_from_url(
          args.resume, map_location='cpu', check_hash = True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')

    # this load the weights and model state into the model
    model.load_state_dict(checkpoint['model'], strict=True)

In [ ]:
letter = "b"
the_image = PIL.Image.open('/kaggle/input/2021-spring-coml-face-recognition-competition/' + str(letter) + '.jpg')
width, height = the_image.size

if width > 1600 and height > 1600:
  new_size = (1600, 1600)
  the_image = the_image.resize(new_size)

scores, boxes = detect(the_image, model, transform)

bbox_dim_list = bbox_dims(the_image, scores, boxes, plot_classes)

# Crop Images, this will be fed into the coml generator
cropped_images = []
for bbox in bbox_dim_list:
    cropped_images.append(the_image.crop((bbox[0], bbox[1], bbox[2], bbox[3]))) # Left, upper, right, bottom

myEmbeddings = np.empty((metadata.shape[0], 128))

for i,img in enumerate(cropped_images):
  img = align_image(np.asarray(img))
  img = img.astype('float32')
  img =img / 255.0
  img = np.expand_dims(img, axis=0)
  person = str(i)
  myEmbeddings[i] = base_model.predict(img)
plotWithNameAndId(the_image, scores, boxes, plot_classes, cIdsB, 'b')

In [ ]:
letter = "c"
the_image = PIL.Image.open('/kaggle/input/2021-spring-coml-face-recognition-competition/' + str(letter) + '.jpg')
width, height = the_image.size

if width > 1600 and height > 1600:
  new_size = (1600, 1600)
  the_image = the_image.resize(new_size)

scores, boxes = detect(the_image, model, transform)

bbox_dim_list = bbox_dims(the_image, scores, boxes, plot_classes)

# Crop Images, this will be fed into the coml generator
cropped_images = []
for bbox in bbox_dim_list:
    cropped_images.append(the_image.crop((bbox[0], bbox[1], bbox[2], bbox[3]))) # Left, upper, right, bottom

myEmbeddings = np.empty((metadata.shape[0], 128))

for i,img in enumerate(cropped_images):
  img = align_image(np.asarray(img))
  img = img.astype('float32')
  img =img / 255.0
  img = np.expand_dims(img, axis=0)
  person = str(i)
  myEmbeddings[i] = base_model.predict(img)
plotWithNameAndId(the_image, scores, boxes, plot_classes, cIdsC, 'c')

In [ ]:
letter = "d"
the_image = PIL.Image.open('/kaggle/input/2021-spring-coml-face-recognition-competition/' + str(letter) + '.jpg')
width, height = the_image.size

if width > 1600 and height > 1600:
  new_size = (1600, 1600)
  the_image = the_image.resize(new_size)

scores, boxes = detect(the_image, model, transform)

bbox_dim_list = bbox_dims(the_image, scores, boxes, plot_classes)

# Crop Images, this will be fed into the coml generator
cropped_images = []
for bbox in bbox_dim_list:
    cropped_images.append(the_image.crop((bbox[0], bbox[1], bbox[2], bbox[3]))) # Left, upper, right, bottom

myEmbeddings = np.empty((metadata.shape[0], 128))

for i,img in enumerate(cropped_images):
  img = align_image(np.asarray(img))
  img = img.astype('float32')
  img =img / 255.0
  img = np.expand_dims(img, axis=0)
  person = str(i)
  myEmbeddings[i] = base_model.predict(img)
plotWithNameAndId(the_image, scores, boxes, plot_classes, cIdsD, 'd')

In [ ]:
os.chdir('/kaggle/working')
with open('submissions.csv', 'a') as csvfile:
    file = open("submissions.csv")
    reader = csv.reader(file)
    filewriter = csv.writer(csvfile, delimiter=',', quotechar = "|", quoting = csv.QUOTE_MINIMAL)
    for i in cIdsA:
        if not i in IdsUsed:
            lines = len(list(reader))
            if lines < 57:
                filewriter.writerow([i, int(0), int(0), int(0), int(0)])
    for i in cIdsB:
        if not i in IdsUsed:
            lines = len(list(reader))
            if lines < 57:
                filewriter.writerow([i, int(0), int(0), int(0), int(0)])
    for i in cIdsC:
        if not i in IdsUsed:
            lines = len(list(reader))
            if lines < 57:
                filewriter.writerow([i, int(0), int(0), int(0), int(0)])
    for i in cIdsD:
        if not i in IdsUsed:
            lines = len(list(reader))
            if lines < 57:
                filewriter.writerow([i, int(0), int(0), int(0), int(0)])